# PyGreentea Network Generator 

### Load the dependencies

In [1]:
%matplotlib inline

from __future__ import print_function
import h5py
import numpy as np
from numpy import float32, int32, uint8, dtype
import sys
import matplotlib.pyplot as plt


pygt_path = '../PyGreentea'
import sys, os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), pygt_path))

import math

import PyGreentea as pygt

### Load the default network template

In [2]:
netconf = pygt.netgen.NetConf()

### Set the memory limits for the GPU

In [11]:
# We use cuDNN, so:
netconf.use_batchnorm = False
netconf.dropout = 0.0
netconf.ignore_conv_buffer = True
# 4 GB total, ignore convolution buffer. Let's keep 0.5 GB for implementation dependent buffers.
netconf.mem_global_limit = 3.5 * 1024 * 1024 * 1024
# 4 GB convolution buffer limit
netconf.mem_buf_limit = 3.5 * 1024 * 1024 * 1024
# netconf.unet_downsampling_strategy = [[3,3,3],[3,3,3],[3,3,3]]
netconf.fmap_start = 20
netconf.fmap_inc = 1
netconf.fmap_dec = 1
netconf.unet_depth = 4
netconf.unet_downsampling_strategy = [[2,2,2],[2,2,2],[2,2,2],[1,1,1]]

### Explore possible network input/output shapes for the chosen settings

In [12]:
# We test memory usage for training
mode = pygt.netgen.caffe_pb2.TRAIN
# The minimum we're interested in
shape_min = [100,100,100]
# And maximum
shape_max = [200,200,200]
# We want Z and Y to be independent, but X == Y
constraints = [None, None, lambda x: x[1]]
# Compute (can be quite intensive)
inshape, outshape, fmaps = pygt.netgen.compute_valid_io_shapes(netconf,mode,shape_min,shape_max,constraints=constraints)

-- Invalid: [100] => []
-- Invalid: [101] => []
-- Invalid: [102] => []
-- Invalid: [103] => []
-- Invalid: [104] => []
-- Invalid: [105] => []
-- Invalid: [106] => []
-- Invalid: [107] => []
-- Invalid: [108] => []
-- Invalid: [109] => []
-- Invalid: [110] => []
-- Invalid: [111] => []
-- Invalid: [112] => []
-- Invalid: [113] => []
-- Invalid: [114] => []
-- Invalid: [115] => []
-- Invalid: [116] => []
-- Invalid: [117] => []
-- Invalid: [118] => []
-- Invalid: [119] => []
-- Invalid: [120] => []
-- Invalid: [121] => []
-- Invalid: [122] => []
-- Invalid: [123] => []
-- Invalid: [124] => []
-- Invalid: [125] => []
-- Invalid: [126] => []
-- Invalid: [127] => []
-- Invalid: [128] => []
-- Invalid: [129] => []
-- Invalid: [130] => []
-- Invalid: [131] => []
++++ Valid: [132] => [8]
-- Invalid: [133] => []
-- Invalid: [134] => []
-- Invalid: [135] => []
-- Invalid: [136] => []
-- Invalid: [137] => []
-- Invalid: [138] => []
-- Invalid: [139] => []
++++ Valid: [140] => [16]
-- Invalid: [

In [30]:
i=0
for o in zip(inshape,outshape,fmaps):
    print('i=' + str(i))
    print(o)
    i+=1

i=0
([132, 132, 132], [8, 8, 8], 90)
i=1
([140, 132, 132], [16, 8, 8], 85)
i=2
([148, 132, 132], [24, 8, 8], 80)
i=3
([156, 132, 132], [32, 8, 8], 75)
i=4
([164, 132, 132], [40, 8, 8], 71)
i=5
([172, 132, 132], [48, 8, 8], 68)
i=6
([180, 132, 132], [56, 8, 8], 65)
i=7
([188, 132, 132], [64, 8, 8], 62)
i=8
([196, 132, 132], [72, 8, 8], 59)
i=9
([132, 140, 140], [8, 16, 16], 79)
i=10
([140, 140, 140], [16, 16, 16], 74)
i=11
([148, 140, 140], [24, 16, 16], 69)
i=12
([156, 140, 140], [32, 16, 16], 65)
i=13
([164, 140, 140], [40, 16, 16], 62)
i=14
([172, 140, 140], [48, 16, 16], 59)
i=15
([180, 140, 140], [56, 16, 16], 56)
i=16
([188, 140, 140], [64, 16, 16], 53)
i=17
([196, 140, 140], [72, 16, 16], 51)
i=18
([132, 148, 148], [8, 24, 24], 70)
i=19
([140, 148, 148], [16, 24, 24], 65)
i=20
([148, 148, 148], [24, 24, 24], 61)
i=21
([156, 148, 148], [32, 24, 24], 57)
i=22
([164, 148, 148], [40, 24, 24], 54)
i=23
([172, 148, 148], [48, 24, 24], 51)
i=24
([180, 148, 148], [56, 24, 24], 48)
i=25
(

### Visualization

In [ ]:
plt.figure()
# Combined output size versus feature map count
plt.scatter([x[0]*x[1]*x[2] for x in outshape], fmaps, alpha = 0.5)
plt.ylabel('Feature maps')
plt.xlabel('Combined output size')
plt.show()

### Pick parameters, actually generate and store the network

In [31]:
netconf.input_shape = inshape[40]
netconf.output_shape = outshape[40]
netconf.fmap_start = fmaps[40]

print ('Input shape: %s' % netconf.input_shape)
print ('Output shape: %s' % netconf.output_shape)
print ('Feature maps: %s' % netconf.fmap_start)

netconf.loss_function = "euclid"
train_net_conf_euclid, test_net_conf = pygt.netgen.create_nets(netconf)
netconf.loss_function = "malis"
train_net_conf_malis, test_net_conf = pygt.netgen.create_nets(netconf)

with open('net_train_euclid.prototxt', 'w') as f:
    print(train_net_conf_euclid, file=f)
with open('net_train_malis.prototxt', 'w') as f:
    print(train_net_conf_malis, file=f)
with open('net_test.prototxt', 'w') as f:
    print(test_net_conf, file=f)

Input shape: [164, 164, 164]
Output shape: [40, 40, 40]
Feature maps: 41
f: 1 w: [164, 164, 164] d: [1, 1, 1]
WM: 0
CM: 0
AM: 0
f: 41 w: [162, 162, 162] d: [1, 1, 1]
WM: 4428
CM: 476381952
AM: 0
f: 41 w: [160, 160, 160] d: [1, 1, 1]
WM: 181548
CM: 18825765984
AM: 0
f: 41 w: [80, 80, 80] d: [1, 1, 1]
WM: 0
CM: 0
AM: 0
f: 41 w: [78, 78, 78] d: [1, 1, 1]
WM: 181548
CM: 2267136000
AM: 0
f: 41 w: [76, 76, 76] d: [1, 1, 1]
WM: 181548
CM: 2101316256
AM: 0
f: 41 w: [38, 38, 38] d: [1, 1, 1]
WM: 0
CM: 0
AM: 0
f: 41 w: [36, 36, 36] d: [1, 1, 1]
WM: 181548
CM: 242973216
AM: 0
f: 41 w: [34, 34, 34] d: [1, 1, 1]
WM: 181548
CM: 206592768
AM: 0
f: 41 w: [17, 17, 17] d: [1, 1, 1]
WM: 0
CM: 0
AM: 0
f: 41 w: [15, 15, 15] d: [1, 1, 1]
WM: 181548
CM: 21754764
AM: 0
f: 41 w: [13, 13, 13] d: [1, 1, 1]
WM: 181548
CM: 14944500
AM: 0
f: 41 w: [13, 13, 13] d: [1, 1, 1]
WM: 0
CM: 0
AM: 0
f: 41 w: [11, 11, 11] d: [1, 1, 1]
WM: 181548
CM: 9728316
AM: 0
f: 41 w: [9, 9, 9] d: [1, 1, 1]
WM: 181548
CM: 5893668
AM: 0
f